#### clover api (OCR)

* google api 계정 생성 방법 (https://www.youtube.com/watch?v=hkKKfEqZvn4)

* 실행하기 전 poppler 다운로드 (https://github.com/oschwartz10612/poppler-windows/releases)
* 해당경로에 압축해제 (C:\)


In [8]:
from google.cloud import vision
import os
from pdf2image import convert_from_path
import io

# 서비스 계정 키 파일 경로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:/Users/User/Desktop/eduSmile-master/ai_models/ocr_google_api_key.json"

def extract_text_from_pdf(pdf_path):
    # Vision API 클라이언트 생성
    client = vision.ImageAnnotatorClient()
    
    # PDF를 이미지로 변환
    images = convert_from_path(pdf_path, poppler_path=r'C:\poppler-24.08.0\Library\bin') 
    # images = convert_from_path(pdf_path)
    
    all_text = ""
    
    # 각 페이지별로 처리
    for i, image in enumerate(images):
        # 이미지를 바이트로 변환
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='PNG')
        content = img_byte_arr.getvalue()
        
        # Vision API 이미지 객체 생성
        image = vision.Image(content=content)
        
        # 텍스트 감지
        response = client.text_detection(image=image)
        texts = response.text_annotations
        
        # 결과 처리
        if texts:
            # 첫 번째 항목이 전체 텍스트를 포함
            all_text += f"\n\n=== Page {i+1} ===\n"
            all_text += texts[0].description
            
    return all_text



In [ ]:
import os

# 사용 예시
pdf_path = r"C:\Users\User\Desktop\eduSmile-master\ai_models\이력서(자기소개).pdf"  # PDF 파일 경로
output_path = r"C:\Users\User\Desktop\eduSmile-master\ai_models\pdf2text"  # 저장될 경로

try:
    extracted_text = extract_text_from_pdf(pdf_path)
    print("추출된 텍스트:")
    print(extracted_text)
    
    # 원본 PDF 파일의 이름만 가져오기
    pdf_filename = os.path.basename(pdf_path)
    # PDF 확장자를 txt로 변경
    txt_filename = pdf_filename.rsplit('.', 1)[0] + '.txt'
    # 출력 경로와 파일명 결합
    output_filename = os.path.join(output_path, txt_filename)
    
    # 출력 디렉토리가 없으면 생성
    os.makedirs(output_path, exist_ok=True)
    
    # 텍스트 파일로 저장
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(extracted_text)
        
except Exception as e:
    print(f"에러 발생: {str(e)}")

#### txt2summary (Claude)

In [18]:
from google.cloud import vision
import os
from pdf2image import convert_from_path
import io
from anthropic import Anthropic
from datetime import datetime
from pathlib import Path
import json
import time

class PDFProcessor:
    def __init__(self, api_key_path: str, output_folder: str = "pdf2summary"):
        """PDF와 학생 정보를 처리하는 클래스"""
        self.output_folder = Path(output_folder)
        self.api_key = self._read_api_key(api_key_path)
        self.claude_client = Anthropic(api_key=self.api_key)
        self.vision_client = vision.ImageAnnotatorClient()
        self._create_folders()
        self.last_api_call = 0
        self.min_delay = 1

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """PDF에서 텍스트를 추출합니다."""
        # PDF를 이미지로 변환
        images = convert_from_path(pdf_path, poppler_path=r'C:\poppler-24.08.0\Library\bin')
        
        all_text = ""
        
        # 각 페이지별로 처리
        for i, image in enumerate(images):
            # 이미지를 바이트로 변환
            img_byte_arr = io.BytesIO()
            image.save(img_byte_arr, format='PNG')
            content = img_byte_arr.getvalue()
            
            # Vision API 이미지 객체 생성
            image = vision.Image(content=content)
            
            # 텍스트 감지
            response = self.vision_client.text_detection(image=image)
            texts = response.text_annotations
            
            # 결과 처리
            if texts:
                # 첫 번째 항목이 전체 텍스트를 포함
                all_text += f"\n\n=== Page {i+1} ===\n"
                all_text += texts[0].description
                
        return all_text

    def _read_api_key(self, api_key_path: str) -> str:
        """API 키를 파일에서 읽어옵니다."""
        try:
            with open(api_key_path, 'r', encoding='utf-8') as f:
                return f.read().strip()
        except Exception as e:
            raise Exception(f"API 키 파일을 읽는 중 오류 발생: {str(e)}")

    def _create_folders(self):
        """필요한 폴더를 생성합니다."""
        self.output_folder.mkdir(parents=True, exist_ok=True)

    def _generate_filename(self, original_filename: str) -> str:
        """결과 파일명을 생성합니다."""
        date_str = datetime.now().strftime('%Y%m%d_%H%M%S')
        original_name = Path(original_filename).stem
        return f"summary_{original_name}_{date_str}"

    def _respect_rate_limit(self):
        """API 호출 간격을 조절합니다."""
        current_time = time.time()
        time_since_last_call = current_time - self.last_api_call
        if time_since_last_call < self.min_delay:
            time.sleep(self.min_delay - time_since_last_call)
        self.last_api_call = time.time()

    def get_claude_response(self, prompt: str) -> str:
        """Claude API를 호출하여 응답을 받습니다."""
        self._respect_rate_limit()
        try:
            response = self.claude_client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=2000,
                temperature=0.7,
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            )
            if isinstance(response.content, list):
                if len(response.content) > 0 and hasattr(response.content[0], 'text'):
                    return response.content[0].text
                return ' '.join(str(item) for item in response.content)
            elif hasattr(response.content, 'text'):
                return response.content.text
            return str(response.content)
        except Exception as e:
            print(f"API 호출 중 오류 발생: {str(e)}")
            raise

    def analyze_student_info(self, text_content: str) -> dict:
        """Claude API를 사용하여 학생 정보를 분석합니다."""
        prompt = f"""
        다음 텍스트에서 학생 정보를 추출하여 JSON 형식으로 정리해주세요.
        반드시 아래의 JSON 형식만을 사용하여 응답해주세요.
        추가 설명이나 다른 텍스트는 포함하지 마세요.

        {{
            "과목별_성적": {{
                "과목명": "점수",
                ...
            }},
            "출결_상황": {{
                "출석일수": "",
                "결석일수": "",
                "지각": "",
                "조퇴": ""
            }},
            "특기사항": [],
            "가족사항": [],
            "기타정보": {{}}
        }}

        텍스트 내용:
        {text_content}
        """

        try:
            response_text = self.get_claude_response(prompt)
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            
            if json_start == -1 or json_end == 0:
                raise ValueError("JSON 형식을 찾을 수 없습니다")
                
            json_str = response_text[json_start:json_end]
            info = json.loads(json_str)
            
            if not isinstance(info, dict):
                raise ValueError("잘못된 JSON 구조")
            
            return info
            
        except json.JSONDecodeError as e:
            print(f"JSON 파싱 오류: {str(e)}")
            print(f"응답 텍스트: {response_text}")
            raise
        except Exception as e:
            print(f"정보 분석 중 오류 발생: {str(e)}")
            raise

    def process_pdf(self, pdf_path: str) -> dict:
        """PDF를 처리하고 결과를 저장합니다."""
        try:
            # PDF에서 텍스트 추출
            print("PDF에서 텍스트 추출 중...")
            extracted_text = self.extract_text_from_pdf(pdf_path)
            print("텍스트 추출 완료")

            # Claude API를 사용하여 정보 분석
            print("\n텍스트 분석 중...")
            student_info = self.analyze_student_info(extracted_text)
            
            # 결과 파일 저장
            output_filename = self._generate_filename(os.path.basename(pdf_path))
            output_path = self.output_folder / f"{output_filename}.json"
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(student_info, f, ensure_ascii=False, indent=2)

            return {
                "status": "success",
                "파일명": os.path.basename(pdf_path),
                "저장경로": str(output_path),
                "추출된_정보": student_info
            }

        except Exception as e:
            return {
                "status": "error",
                "파일명": os.path.basename(pdf_path),
                "오류": str(e)
            }



In [ ]:
# 경로 설정
pdf_path = r"C:\Users\User\Desktop\eduSmile-master\ai_models\이력서(자기소개).pdf"
api_key_path = r"C:\Users\User\Desktop\eduSmile-master\ai_models\claude_api_key.txt"
summary_output_folder = r"C:\Users\User\Desktop\eduSmile-master\ai_models\pdf2summary"

try:
    # Google Vision API 자격 증명 설정
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:/Users/User/Desktop/eduSmile-master/ai_models/ocr_google_api_key.json"

    # PDF 처리 및 분석
    processor = PDFProcessor(api_key_path, summary_output_folder)
    results = processor.process_pdf(pdf_path)

    # 결과 출력
    if results["status"] == "success":
        print(f"\n처리 완료: {results['파일명']}")
        print(f"저장 경로: {results['저장경로']}")
        print("\n추출된 정보:")
        print(json.dumps(results['추출된_정보'], ensure_ascii=False, indent=2))
    else:
        print(f"\n처리 실패: {results['파일명']}")
        print(f"오류: {results['오류']}")

except Exception as e:
    print(f"프로그램 실행 중 오류 발생: {str(e)}")

